In [567]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from IPython.display import display
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import MultiLabelBinarizer


In [568]:
df =pd.read_csv("action.csv")

In [569]:
print(df.index)

RangeIndex(start=0, stop=52452, step=1)


In [570]:
df.head()

,movie_id,movie_name,year,certificate,runtime,genre,rating,description,director,director_id,star,star_id,votes,gross(in $)
0,tt9114286,Black Panther: Wakanda Forever,2022,PG-13,161 min,"Action, Adventure, Drama",6.9,The people of Wakanda fight to protect their h...,Ryan Coogler,/name/nm3363032/,"Letitia Wright, \nLupita Nyong'o, \nDanai Guri...","/name/nm4004793/,/name/nm2143282/,/name/nm1775...",204835.0,NaN
1,tt1630029,Avatar: The Way of Water,2022,PG-13,192 min,"Action, Adventure, Fantasy",7.8,Jake Sully lives with his newfound family form...,James Cameron,/name/nm0000116/,"Sam Worthington, \nZoe Saldana, \nSigourney We...","/name/nm0941777/,/name/nm0757855/,/name/nm0000...",295119.0,NaN
2,tt5884796,Plane,2023,R,107 min,"Action, Thriller",6.5,A pilot finds himself caught in a war zone aft...,Jean-François Richet,/name/nm0724938/,"Gerard Butler, \nMike Colter, \nTony Goldwyn, ...","/name/nm0124930/,/name/nm1591496/,/name/nm0001...",26220.0,NaN
3,tt6710474,Everything Everywhere All at Once,2022,R,139 min,"Action, Adventure, Comedy",8.0,A middle-aged Chinese immigrant is swept up in...,"Dan Kwan, \nDaniel Scheinert",/name/nm3453283/,"Michelle Yeoh, \nStephanie Hsu, \nJamie Lee Cu...","/name/nm3215397/,/name/nm0000706/,/name/nm3513...",327858.0,NaN
4,tt5433140,Fast X,2023,NaN,NaN,"Action, Crime, Mystery",NaN,Dom Toretto and his family are targeted by the...,Louis Leterrier,/name/nm0504642/,"Vin Diesel, \nJordana Brewster, \nTyrese Gibso...","/name/nm0004874/,/name/nm0108287/,/name/nm0879...",NaN,NaN


In [571]:
df = df.drop(columns=["movie_id", "movie_name", "description","director","star"])

In [572]:
df=df.dropna(subset=['rating'])

In [573]:
df.head()

,year,certificate,runtime,genre,rating,director_id,star_id,votes,gross(in $)
0,2022,PG-13,161 min,"Action, Adventure, Drama",6.9,/name/nm3363032/,"/name/nm4004793/,/name/nm2143282/,/name/nm1775...",204835.0,NaN
1,2022,PG-13,192 min,"Action, Adventure, Fantasy",7.8,/name/nm0000116/,"/name/nm0941777/,/name/nm0757855/,/name/nm0000...",295119.0,NaN
2,2023,R,107 min,"Action, Thriller",6.5,/name/nm0724938/,"/name/nm0124930/,/name/nm1591496/,/name/nm0001...",26220.0,NaN
3,2022,R,139 min,"Action, Adventure, Comedy",8.0,/name/nm3453283/,"/name/nm3215397/,/name/nm0000706/,/name/nm3513...",327858.0,NaN
5,2023,PG-13,125 min,"Action, Adventure, Comedy",6.6,/name/nm0715636/,"/name/nm0748620/,/name/nm1431940/,/name/nm3718...",5396.0,NaN


In [574]:
df.dtypes

year            object
certificate     object
runtime         object
genre           object
rating         float64
director_id     object
star_id         object
votes          float64
gross(in $)    float64
dtype: object

In [575]:
unique_genres = set(genre.strip() for genres in df['genre'] for genre in genres.split(','))
print(unique_genres)

{'Family', 'Fantasy', 'Music', 'Drama', 'Adventure', 'Horror', 'Musical', 'Crime', 'Action', 'Mystery', 'History', 'Biography', 'Romance', 'Comedy', 'Film-Noir', 'Animation', 'Thriller', 'Sport', 'Sci-Fi', 'Western', 'Adult', 'War'}


In [576]:
df['genre'] = df['genre'].apply(lambda x: x.split(', '))

# Używamy MultiLabelBinarizer
mlb = MultiLabelBinarizer()
binarized_data = mlb.fit_transform(df['genre'])

# Tworzymy nowy DataFrame z zerami i jedynkami
binarized_df = pd.DataFrame(binarized_data, columns=mlb.classes_)

# Wyświetlamy wynik
binarized_df.head()


,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,1,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [577]:
binarized_df.fillna(0,inplace=True)

In [578]:
df = df.drop(columns=["genre"])

In [579]:
binarized_df.dtypes

Action       int32
Adult        int32
Adventure    int32
Animation    int32
Biography    int32
Comedy       int32
Crime        int32
Drama        int32
Family       int32
Fantasy      int32
Film-Noir    int32
History      int32
Horror       int32
Music        int32
Musical      int32
Mystery      int32
Romance      int32
Sci-Fi       int32
Sport        int32
Thriller     int32
War          int32
Western      int32
dtype: object

In [580]:
print(len(df))

29448


In [581]:
print(len(binarized_df))

29448


In [582]:
print(df.index.equals(binarized_df.index))  # Sprawdza, czy indeksy są identyczne
print(df.index.difference(binarized_df.index))  # Wiersze, które są w df, ale nie w binarized_df
print(binarized_df.index.difference(df.index))  # Wiersze, które są w binarized_df, ale nie w df


False
Index([29448, 29449, 29451, 29452, 29454, 29458, 29459, 29460, 29461, 29462,
       ...
       52318, 52320, 52341, 52342, 52387, 52393, 52404, 52413, 52419, 52421],
      dtype='int64', length=5495)
Index([    4,    24,    29,    31,    36,    37,    41,    54,    59,    64,
       ...
       29423, 29424, 29429, 29437, 29439, 29441, 29443, 29444, 29446, 29447],
      dtype='int64', length=5495)


In [583]:
binarized_df = binarized_df.set_index(df.index)


In [584]:
 df=pd.concat([df, binarized_df], axis = 1)

In [585]:
print(len(df))

29448


In [586]:
filled4y= df['rating'].isna()
print(filled4y)

0        False
1        False
2        False
3        False
5        False
         ...  
52393    False
52404    False
52413    False
52419    False
52421    False
Name: rating, Length: 29448, dtype: bool


In [587]:
binarized_df.head()

,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,1,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [588]:
df.head()

,year,certificate,runtime,rating,director_id,star_id,votes,gross(in $),Action,Adult,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,2022,PG-13,161 min,6.9,/name/nm3363032/,"/name/nm4004793/,/name/nm2143282/,/name/nm1775...",204835.0,NaN,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2022,PG-13,192 min,7.8,/name/nm0000116/,"/name/nm0941777/,/name/nm0757855/,/name/nm0000...",295119.0,NaN,1,0,...,0,0,0,0,0,0,0,0,0,0
2,2023,R,107 min,6.5,/name/nm0724938/,"/name/nm0124930/,/name/nm1591496/,/name/nm0001...",26220.0,NaN,1,0,...,0,0,0,0,0,0,0,1,0,0
3,2022,R,139 min,8.0,/name/nm3453283/,"/name/nm3215397/,/name/nm0000706/,/name/nm3513...",327858.0,NaN,1,0,...,0,0,0,0,0,0,0,0,0,0
5,2023,PG-13,125 min,6.6,/name/nm0715636/,"/name/nm0748620/,/name/nm1431940/,/name/nm3718...",5396.0,NaN,1,0,...,0,0,0,0,0,0,0,0,0,0


In [589]:
filled9= df['rating'].isna()
print(filled9)

0        False
1        False
2        False
3        False
5        False
         ...  
52393    False
52404    False
52413    False
52419    False
52421    False
Name: rating, Length: 29448, dtype: bool


In [590]:
X = df.drop('rating', axis=1) 
y = pd.DataFrame(df, columns=['rating'])

In [591]:
X.head()

,year,certificate,runtime,director_id,star_id,votes,gross(in $),Action,Adult,Adventure,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,2022,PG-13,161 min,/name/nm3363032/,"/name/nm4004793/,/name/nm2143282/,/name/nm1775...",204835.0,NaN,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2022,PG-13,192 min,/name/nm0000116/,"/name/nm0941777/,/name/nm0757855/,/name/nm0000...",295119.0,NaN,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2023,R,107 min,/name/nm0724938/,"/name/nm0124930/,/name/nm1591496/,/name/nm0001...",26220.0,NaN,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,2022,R,139 min,/name/nm3453283/,"/name/nm3215397/,/name/nm0000706/,/name/nm3513...",327858.0,NaN,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5,2023,PG-13,125 min,/name/nm0715636/,"/name/nm0748620/,/name/nm1431940/,/name/nm3718...",5396.0,NaN,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [592]:
X.dtypes

year            object
certificate     object
runtime         object
director_id     object
star_id         object
votes          float64
gross(in $)    float64
Action           int32
Adult            int32
Adventure        int32
Animation        int32
Biography        int32
Comedy           int32
Crime            int32
Drama            int32
Family           int32
Fantasy          int32
Film-Noir        int32
History          int32
Horror           int32
Music            int32
Musical          int32
Mystery          int32
Romance          int32
Sci-Fi           int32
Sport            int32
Thriller         int32
War              int32
Western          int32
dtype: object

In [593]:
y.head()

,rating
0,6.9
1,7.8
2,6.5
3,8.0
5,6.6


In [594]:
le = LabelEncoder()
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = le.fit_transform(X[col].astype(str))

In [595]:
X.dtypes

year             int32
certificate      int32
runtime          int32
director_id      int32
star_id          int32
votes          float64
gross(in $)    float64
Action           int32
Adult            int32
Adventure        int32
Animation        int32
Biography        int32
Comedy           int32
Crime            int32
Drama            int32
Family           int32
Fantasy          int32
Film-Noir        int32
History          int32
Horror           int32
Music            int32
Musical          int32
Mystery          int32
Romance          int32
Sci-Fi           int32
Sport            int32
Thriller         int32
War              int32
Western          int32
dtype: object

In [596]:
nan = X['gross(in $)'].isna().sum()
print(nan)

26688


In [597]:
filled= X['gross(in $)'].count()
print(filled)

2760


In [598]:
X.fillna({'gross(in $)':X['gross(in $)'].median()}, inplace=True)
X['gross(in $)'].head()

0    10903645.5
1    10903645.5
2    10903645.5
3    10903645.5
5    10903645.5
Name: gross(in $), dtype: float64

In [599]:
czy_nan_w_df = X.isna().any().any()
print(czy_nan_w_df)  # True, jeśli gdziekolwiek w DF jest NaN


False


In [600]:
czy_nan_w_df2 = y.isna().any().any()
print(czy_nan_w_df2)  # True, jeśli gdziekolwiek w DF jest NaN

False


In [601]:
filled3= y['rating'].isna().sum()
print(filled3)

0


In [602]:
y.fillna({'rating':y['rating'].median()}, inplace=True)
y['rating'].head()

0    6.9
1    7.8
2    6.5
3    8.0
5    6.6
Name: rating, dtype: float64

In [603]:
kolumny_z_nan = X.isna().any()
print(kolumny_z_nan)  # Lista kolumn z NaN


year           False
certificate    False
runtime        False
director_id    False
star_id        False
votes          False
gross(in $)    False
Action         False
Adult          False
Adventure      False
Animation      False
Biography      False
Comedy         False
Crime          False
Drama          False
Family         False
Fantasy        False
Film-Noir      False
History        False
Horror         False
Music          False
Musical        False
Mystery        False
Romance        False
Sci-Fi         False
Sport          False
Thriller       False
War            False
Western        False
dtype: bool


In [604]:
print(type(kolumny_z_nan))

<class 'pandas.core.series.Series'>


In [605]:
X.dtypes

year             int32
certificate      int32
runtime          int32
director_id      int32
star_id          int32
votes          float64
gross(in $)    float64
Action           int32
Adult            int32
Adventure        int32
Animation        int32
Biography        int32
Comedy           int32
Crime            int32
Drama            int32
Family           int32
Fantasy          int32
Film-Noir        int32
History          int32
Horror           int32
Music            int32
Musical          int32
Mystery          int32
Romance          int32
Sci-Fi           int32
Sport            int32
Thriller         int32
War              int32
Western          int32
dtype: object

In [606]:
y.head()

,rating
0,6.9
1,7.8
2,6.5
3,8.0
5,6.6


In [607]:
X['director_id'] = X['director_id'].astype('category').cat.codes

In [608]:
le = LabelEncoder()
X["director_id"] = le.fit_transform(X["director_id"])


In [609]:
X.head()

,year,certificate,runtime,director_id,star_id,votes,gross(in $),Action,Adult,Adventure,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,113,15,64,11718,25719,204835.0,10903645.5,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,113,15,95,8,17912,295119.0,10903645.5,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,114,17,9,5320,6144,26220.0,10903645.5,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,113,17,41,11835,24593,327858.0,10903645.5,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5,114,15,27,5263,15336,5396.0,10903645.5,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [610]:
y.head()

,rating
0,6.9
1,7.8
2,6.5
3,8.0
5,6.6


In [611]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29448 entries, 0 to 52421
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   year         29448 non-null  int32  
 1   certificate  29448 non-null  int32  
 2   runtime      29448 non-null  int32  
 3   director_id  29448 non-null  int64  
 4   star_id      29448 non-null  int32  
 5   votes        29448 non-null  float64
 6   gross(in $)  29448 non-null  float64
 7   Action       29448 non-null  int32  
 8   Adult        29448 non-null  int32  
 9   Adventure    29448 non-null  int32  
 10  Animation    29448 non-null  int32  
 11  Biography    29448 non-null  int32  
 12  Comedy       29448 non-null  int32  
 13  Crime        29448 non-null  int32  
 14  Drama        29448 non-null  int32  
 15  Family       29448 non-null  int32  
 16  Fantasy      29448 non-null  int32  
 17  Film-Noir    29448 non-null  int32  
 18  History      29448 non-null  int32  
 19  Horror   

In [612]:
X.describe()

,year,certificate,runtime,director_id,star_id,votes,gross(in $),Action,Adult,Adventure,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
count,29448.000000,29448.000000,29448.000000,29448.000000,29448.000000,2.944800e+04,2.944800e+04,29448.000000,29448.000000,29448.000000,...,29448.00000,29448.000000,29448.000000,29448.000000,29448.000000,29448.000000,29448.000000,29448.000000,29448.000000,29448.000000
mean,85.897786,22.151453,153.281819,6352.416599,14636.567747,1.159460e+04,1.378710e+07,0.998506,0.000034,0.206398,...,0.05036,0.006045,0.006384,0.021462,0.055250,0.054910,0.007505,0.132742,0.021767,0.014127
std,23.130385,7.047892,97.499476,4224.572174,8444.372621,7.039608e+04,2.617441e+07,0.038626,0.005827,0.404726,...,0.21869,0.077513,0.079647,0.144920,0.228471,0.227809,0.086306,0.339302,0.145925,0.118015
min,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000e+00,7.000000e+00,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,72.000000,16.000000,36.000000,2867.000000,7322.750000,2.975000e+01,1.090365e+07,1.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,92.000000,28.000000,220.000000,5742.000000,14644.500000,1.320000e+02,1.090365e+07,1.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,105.000000,28.000000,232.000000,9701.250000,21945.250000,9.442500e+02,1.090365e+07,1.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,115.000000,28.000000,241.000000,15100.000000,29131.000000,2.675531e+06,9.366622e+08,1.000000,1.000000,1.000000,...,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [613]:
scaler = StandardScaler()

In [614]:
numeric_cols = X.select_dtypes(include=['number']).columns

In [615]:
numeric_cols

Index(['year', 'certificate', 'runtime', 'director_id', 'star_id', 'votes',
       'gross(in $)', 'Action', 'Adult', 'Adventure', 'Animation', 'Biography',
       'Comedy', 'Crime', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History',
       'Horror', 'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Sport',
       'Thriller', 'War', 'Western'],
      dtype='object')

In [616]:
type(numeric_cols)

pandas.core.indexes.base.Index

In [617]:
numeric_cols=numeric_cols.drop(["director_id", "star_id",'Action', 'Adult', 'Adventure', 'Animation', 'Biography',
       'Comedy', 'Crime', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History',
       'Horror', 'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Sport',
       'Thriller', 'War', 'Western'])

In [618]:
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

In [619]:
X.describe()

,year,certificate,runtime,director_id,star_id,votes,gross(in $),Action,Adult,Adventure,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
count,2.944800e+04,2.944800e+04,2.944800e+04,29448.000000,29448.000000,2.944800e+04,2.944800e+04,29448.000000,29448.000000,29448.000000,...,29448.00000,29448.000000,29448.000000,29448.000000,29448.000000,29448.000000,29448.000000,29448.000000,29448.000000,29448.000000
mean,-2.007510e-16,1.853086e-16,-9.265431e-17,6352.416599,14636.567747,1.544238e-17,-3.860596e-18,0.998506,0.000034,0.206398,...,0.05036,0.006045,0.006384,0.021462,0.055250,0.054910,0.007505,0.132742,0.021767,0.014127
std,1.000017e+00,1.000017e+00,1.000017e+00,4224.572174,8444.372621,1.000017e+00,1.000017e+00,0.038626,0.005827,0.404726,...,0.21869,0.077513,0.079647,0.144920,0.228471,0.227809,0.086306,0.339302,0.145925,0.118015
min,-3.713697e+00,-3.143043e+00,-1.572156e+00,0.000000,0.000000,-1.646369e-01,-5.267485e-01,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-6.008556e-01,-8.728223e-01,-1.202917e+00,2867.000000,7322.750000,-1.642853e-01,-1.101651e-01,1.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.638226e-01,8.298433e-01,6.843043e-01,5742.000000,14644.500000,-1.628328e-01,-1.101651e-01,1.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8.258634e-01,8.298433e-01,8.073840e-01,9701.250000,21945.250000,-1.512943e-01,-1.101651e-01,1.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.258202e+00,8.298433e-01,8.996938e-01,15100.000000,29131.000000,3.784276e+01,3.525928e+01,1.000000,1.000000,1.000000,...,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [620]:
# Podział na zbiór treningowy (60%) i tymczasowy (40%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)

# Podział tymczasowego zbioru na walidacyjny (20%) i testowy (20%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)




In [621]:
# Przykładowe hiperparametry
EMBEDDING_DIM = 10  # Wymiar embeddingu dla star_id i director_id
HIDDEN_DIM = 128
BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 0.001

In [622]:
def get_dataloader(X, y, batch_size=BATCH_SIZE, shuffle=True):
    class MovieDataset(Dataset):
        # Atrybut klasowy
        exclude = ['director_id', 'star_id']
        
        def __init__(self, X, y):
            # Dostęp do atrybutu klasowego za pomocą `self.__class__.exclude`
            self.numeric_features = torch.tensor(X[[col for col in X.columns if col not in self.__class__.exclude]].values)
            self.director_ids = torch.tensor(X['director_id'].values, dtype=torch.long)
            self.star_ids = torch.tensor(X['star_id'].values, dtype=torch.long)
            self.y = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)
        
        def __len__(self):
            return len(self.y)
        
        def __getitem__(self, idx):
            return self.numeric_features[idx], self.director_ids[idx], self.star_ids[idx], self.y[idx]
    
    return DataLoader(MovieDataset(X, y), batch_size=batch_size, shuffle=shuffle)


In [623]:
X.head()

,year,certificate,runtime,director_id,star_id,votes,gross(in $),Action,Adult,Adventure,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,1.171735,-1.014711,-0.915731,11718,25719,2.745092,-0.110165,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1.171735,-1.014711,-0.597776,8,17912,4.027628,-0.110165,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1.214969,-0.730934,-1.479847,5320,6144,0.207762,-0.110165,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1.171735,-0.730934,-1.151634,11835,24593,4.492705,-0.110165,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5,1.214969,-1.014711,-1.295227,5263,15336,-0.088055,-0.110165,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [624]:
train_loader = get_dataloader(X_train, y_train)  # Domyślnie shuffle=True
val_loader = get_dataloader(X_val, y_val)        # Domyślnie shuffle=True
test_loader = get_dataloader(X_test, y_test, shuffle=False)  # Testowy zbiór bez losowego mieszania


In [625]:
def get_model(num_numeric_features, num_directors, num_stars, EMBEDDING_DIM, HIDDEN_DIM=128):
    class MovieRatingModel(nn.Module):
        def __init__(self):
            super().__init__()
            self.director_embedding = nn.Embedding(num_directors, EMBEDDING_DIM)
            self.star_embedding = nn.Embedding(num_stars, EMBEDDING_DIM)
            input_dim = num_numeric_features + 2 * EMBEDDING_DIM  # Poprawione
            print(f"Expected input_dim: {input_dim}")
            self.fc = nn.Sequential(
                nn.Linear(input_dim, HIDDEN_DIM),
                nn.ReLU(),
                nn.Linear(HIDDEN_DIM, 64),
                nn.ReLU(),
                nn.Linear(64, 1)  # Wyjście: jedna wartość (przewidywany rating)
            )

        def forward(self, numeric_features, director_ids, star_ids):
            director_emb = self.director_embedding(director_ids)
            star_emb = self.star_embedding(star_ids)
            numeric_features = numeric_features.float()
            x = torch.cat([numeric_features, director_emb, star_emb], dim=1)
            return self.fc(x)

    return MovieRatingModel()


In [626]:
print( X_train.shape[1])

29


In [627]:
model_first=get_model( X_train.shape[1]-2,X['director_id'].max()+1,X['star_id'].max()+1,EMBEDDING_DIM)

Expected input_dim: 47


In [628]:
def train_model(model, dataloader, epochs=EPOCHS, lr=LEARNING_RATE):

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for numeric, director, star, target in dataloader:
            print(director.min().item(), director.max().item())
            print(star.min().item(), star.max().item())
            break  # Sprawdź tylko jedną iterację

        for numeric, director, star, target in dataloader:
            
            optimizer.zero_grad()
            output = model(numeric, director, star)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader):.4f}")

In [629]:
train_model(model_first,train_loader)

38 14666
132 28794
Epoch 1/10, Loss: 4.8110
217 15100
319 29131
Epoch 2/10, Loss: 1.7303
65 15093
422 29112
Epoch 3/10, Loss: 1.6018
842 15030
1866 28883
Epoch 4/10, Loss: 1.5348
142 14875
713 28467
Epoch 5/10, Loss: 1.4842
55 14927
329 28162
Epoch 6/10, Loss: 1.4244
22 14315
160 28896
Epoch 7/10, Loss: 1.3619
279 15034
131 28993
Epoch 8/10, Loss: 1.2904
122 14317
1704 29131
Epoch 9/10, Loss: 1.2136
24 14144
152 28698
Epoch 10/10, Loss: 1.1344
